In [1]:
import archetypal as ar
import pandas as pd
import os
from path import Path

# to disable, %matplotlib inline
#%matplotlib notebook
%matplotlib inline

ar.config(use_cache=True)

c:\users\mariana\desktop\projects\buildingtechnology\containerfarmep\archetypal_files\archetypal\archetypal\utils.py:117: UserWarning: The TRNSYS path does not exist. Please set the TRNSYS path with the --trnsys-default-folder option
  warnings.warn(


In [2]:
# set climate region to match weatherfile
climate_region = '5a'

In [70]:
all_buildings_test = ['Stand-aloneRetail','Warehouse','StripMall']

all_epochs = ["Pre1980", "Post1980", "New2004"]

energy_header = ['Cooling (kWh/m2)',
                  'Heating (kWh/m2)',
                  'Lights (kWh/m2)',
                  'Equipment (kWh/m2)',
                  'DHW (kWh/m2)',
                  'EUI (kWh/m2)']
sim_header = ['EP','UMI','error']

date_index = pd.date_range(start='2018/01/01', end='2019/1/1', closed='left', freq='H')

In [81]:
fname = Path(climate_region) / f'{climate_region}_test_EP-Results.xlsx'

#set to true if comparing with real umi results
compare_umi = False

dfs = []
xl = pd.ExcelFile(fname,engine='openpyxl')

for bldg in all_buildings_test:
    
    epoch_dfs = []
    
    for sheet in xl.sheet_names:
        if bldg in sheet:
            
            # read building worksheet in excel as a dataframe
            df = pd.read_excel(fname, sheet_name=sheet, header=[0,1], index_col=0, engine='openpyxl')        
           
            # for each epoch
            for epoch in all_epochs:
                         
                # pull out epoch data frame as individual dataframe
                monthly_results = df[epoch]
                
                # read in umi file
                if compare_umi:
                    #####################################################
                    ###### SAM TO REPLACE CODE HERE ######
                    py_umi = pd.DataFrame(0,index=date_index,columns=energy_header) 
                    # this will need to be replaced with a function that reads 
                    # results from pyumi for each building-epoch combo
                    #####################################################
                else:
                    py_umi = pd.DataFrame(0,index=date_index,columns=energy_header)  
                
                
                #reformat umi results
                umi_month = py_umi.groupby(py_umi.index.month).sum()
                umi_month.index.map(str)
                umi = umi_month.append(umi_month.sum().rename('Annual'))
                
                #create error dataframe
                error_df = (umi-monthly_results)/monthly_results
                
                #concat results with umi and error dataframes
                results_all = pd.concat([monthly_results, umi, error_df], 
                                             keys = sim_header, 
                                             names=['Simulation Type','Energy Type'], 
                                             axis = 1)
                #swap the order of the headers and sort
                results_swapped = results_all.swaplevel(axis=1).sort_index(axis=1).loc[:, (energy_header, sim_header)]
                
                #add temporary results to list of epoch results
                epoch_dfs.append(results_swapped)
            
            # concat all dfs
            annual = pd.concat(epoch_dfs, keys=all_epochs, names = ['Epoch','Energy Type','Simulation Type'], axis = 1)
            annual.index.name = 'Month'
            dfs.append(annual)

In [82]:
dfs[1]

Epoch                    Pre1980                                       \
Energy Type     Cooling (kWh/m2)           Heating (kWh/m2)             
Simulation Type               EP UMI error               EP UMI error   
Month                                                                   
1                       0.000000   0   NaN        43.388246   0  -1.0   
2                       0.000000   0   NaN        33.383288   0  -1.0   
3                       0.000000   0   NaN        17.436747   0  -1.0   
4                       0.057420   0  -1.0         8.648693   0  -1.0   
5                       0.185577   0  -1.0         2.338137   0  -1.0   
6                       0.787639   0  -1.0         0.258041   0  -1.0   
7                       1.463282   0  -1.0         0.063243   0  -1.0   
8                       1.108120   0  -1.0         0.149399   0  -1.0   
9                       0.406120   0  -1.0         0.434845   0  -1.0   
10                      0.000000   0   NaN         4.602811   0  -1.0   
11                      0.000000   0   NaN        15.625929   0  -1.0   
12                      0.000000   0   NaN        37.899270   0  -1.0   
Annual                  4.008159   0  -1.0       164.228649   0  -1.0   

Epoch                                                         ...  \
Energy Type     Lights (kWh/m2)           Equipment (kWh/m2)  ...   
Simulation Type              EP UMI error                 EP  ...   
Month                                                         ...   
1                      1.534924   0  -1.0           0.502334  ...   
2                      1.389044   0  -1.0           0.455051  ...   
3                      1.633062   0  -1.0           0.534554  ...   
4                      1.472726   0  -1.0           0.483295  ...   
5                      1.583993   0  -1.0           0.518444  ...   
6                      1.568079   0  -1.0           0.513423  ...   
7                      1.488641   0  -1.0           0.488316  ...   
8                      1.633062   0  -1.0           0.534554  ...   
9                      1.472726   0  -1.0           0.483295  ...   
10                     1.534924   0  -1.0           0.502334  ...   
11                     1.516225   0  -1.0           0.495221  ...   
12                     1.488641   0  -1.0           0.488316  ...   
Annual                18.316049   0  -1.0           5.999137  ...   

Epoch                   New2004                                                \
Energy Type     Lights (kWh/m2) Equipment (kWh/m2)           DHW (kWh/m2)       
Simulation Type           error                 EP UMI error           EP UMI   
Month                                                                           
1                          -1.0           0.502334   0  -1.0            0   0   
2                          -1.0           0.455051   0  -1.0            0   0   
3                          -1.0           0.534554   0  -1.0            0   0   
4                          -1.0           0.483295   0  -1.0            0   0   
5                          -1.0           0.518444   0  -1.0            0   0   
6                          -1.0           0.513423   0  -1.0            0   0   
7                          -1.0           0.488316   0  -1.0            0   0   
8                          -1.0           0.534554   0  -1.0            0   0   
9                          -1.0           0.483295   0  -1.0            0   0   
10                         -1.0           0.502334   0  -1.0            0   0   
11                         -1.0           0.495221   0  -1.0            0   0   
12                         -1.0           0.488316   0  -1.0            0   0   
Annual                     -1.0           5.999137   0  -1.0            0   0   

Epoch                                         
Energy Type           EUI (kWh/m2)            
Simulation Type error           EP UMI error  
Month                                         
1                 